## Why Vector Fitting?

Before converting from frequency to time domain, we need a function that approximates our frequency response. So let's consider this problem as a curve fitting problem.

# The Transfer Function
(Borrows Heavily From Section 2 Vector Fitting, Piero Triverio)

We will use the following convolution to model the system of input $ u(t) $ and impulse response $ h(t) \in \mathbb{R}^{\bar{q}\times\bar{m}} $:

$$ y(t) = \int_{a}^{b} h(t-\tau)u(\tau) \,dx    $$

Using a Laplace transform, we can convert this model into a frequency-domain representation 
$$ Y(s) = H(s)U(s) $$

where $ Y(s) $, $ H(s) $, and $ U(s) $ are the Laplace transforms of $ y(t) $, $ h(t) $, and $ u(t) $, respectively.

$ H(s) $ is referred to as the transfer function of the system. Since $ H(s) $ is a rational function, we can interpret it to be the ratio of the response over the input for a given frequency $ s = j\omega $.

The goal of the vector fitting algorithm is to efficiently find a useful representation of the transfer function $ H(s) $. Let's start by approximating the transfer function as the ratio of two polynomials. We'll denote our approximation as $ \hat{H}(s) $

$$ \hat{H}(s) = \frac{n(s)}{d(s)} = \frac{a_0 + a_1s^{1} + a_2s^{2} + ... + a_ns^{n}}{b_0 + b_1s^{1} + b_2s^{2} + ... + b_ns^{n}}$$

A core assumption of the vector fitting algorithm is that we can arbitrarily reduce the mean square error of the approximation by increasing the degree of the numerator and denominator polynomials and by solving for the coefficients $ a_i $ and $ b_i $. If we have $\bar{k}$ measurements of the transfer function (which we could either obtain through simulation or direct measurement) represented by $ H_k(s) $, we can estimate the mean squared error, $ \epsilon^2 $, as follows:

$$ \epsilon^2 = \frac{1}{\bar{k}}\sum_{k=1}^{k}[H_k - \frac{n(j\omega_k)}{d(j\omega_k)}]^2 $$ 

Which can be rewritten as:

$$ \epsilon^2 = \frac{1}{\bar{k}}\sum_{k=1}^{k}[\frac{H_kd(j\omega_k) - n(j\omega_k)}{d(j\omega_k)}]^2 $$ 

Naturally, the next step is to solve for the coefficients $ a_i $ and $ b_i $ by minimizing the mean squared error. We could do this through a linear least squares method if we neglect the denomintor of the above expression, but there are at least three major issues with this approach:

1. The denominator of the transfer function can vary wildly from iteration to iteration, leading to a non-linear optimization problem.
2. The denominator of the transfer function can vary wildly as a function for frequency, leading to a non-linear optimization problem.
3. Accurate solutions of the transfer function require large powers of $ s $ in the numerator and denominator, which can lead to numerical errors.

But herein lies the problem: due to the fact that the denominator of the transfer function is not linear, we cannot use a linear method of least squares to minimize the error. While we could use a non-linear optimization method to minimize the error, this would be computationally expensive. 

We can solve the first problem by simply "weighting" the error from iteration to iteration.

$$ \epsilon^2 = \frac{1}{\bar{k}}\sum_{k=1}^{k}[\frac{H_kd(j\omega_k)^{curr} - n(j\omega_k)}{d(j\omega_k)^{prev}}]^2 $$ 

As $ d(j\omega_k)^{prev} \rightarrow  d(j\omega_k)^{curr}$ the error will converge to the "true" error expressed earlier.

While this solves the first problem, the second and third problems remain.

# The Vector Fitting Algorithm

The vector fitting algorithm can solve the the problems mentioned above by using a different approximation for the numerator and denominator of the transfer function. Instead of using a polynomial approximation, we will use a pole-residue approximation. This representation is more stable since it does not require large powers of $ s $ in the numerator and denominator. Obviously, this automatically solves the third problem mentioned above.



# Pole-residue Form 
(Borrows heavily from APPROXIMATIONS OF FUNCTIONS BY SETS OF POLES By A. V. HERSHEY)  

We need an alternate representation of the transfer function $ H(s) $. Our goal will be to analytically convert the ratio of two polynomials we used earlier into a form that does not contain large powers of $ s $.

First, consider the ratio of two polynomials $p(z)$ and $q(z)$ where $p(z)$ is of degree $n-1$ and $q(z)$ be of degree $n$

$$ R(z) = \frac{p(z)}{q(z)} $$ 

p(z) is analogous to the numerator of the transfer function $ n(s) $ and q(z) is analogous to the denominator of the transfer function $ d(s) $ with one more degree. (This extra degree will only make the fitting more accurate in the method of polynomials, so we are justified in using it.)

Using partial fraction decomposition, we can rewrite $ R(z) $ as:  

$$ R(z) = \frac{r_n}{z-z_n} + \frac{p'(z)}{q'(z)}$$

$p'(z)$ is now of degree $n-2$ and $q'(z)$ is of degree $n-1$. We can continue this process until we have a representation of $ R(z) $ as a sum of $n$ terms of the form:

$$ R(z) = \frac{r_n}{z-z_n} + \frac{r_{n-1}}{z-z_{n-1}} + ... + \frac{[polynomial\ of\ degree\ 0]}{[polynomial\ of\ degree\ 1]} $$

By definition a polynomial of degree 0 is a constant and a polynomial of degree 1 is a linear polynomial, so we can rewrite that term as:

$$ \frac{[polynomial\ of\ degree\ 0]}{[polynomial\ of\ degree\ 1]} = \frac{k_0}{k_1z - k_2} = \frac{\frac{k_0}{k_1}}{z - \frac{k_2}{k_1}} $$
If we recognize that this term is of the same form as the other n - 1 terms, we can rewrite the entire expression as:

$$ R(z) = \frac{r_n}{z-z_n} + \frac{r_{n-1}}{z-z_{n-1}} + ... +  \frac{r_1}{z - z_1} $$

Or more succintly:  

$$ R(z) = \sum_{i=1}^{n} \frac{r_i}{z-z_i} $$

This is known as a pole-residue representation of a rational function. 

In case the concept of mathematical residue is foreign to you, the end of the notebook contains a brief explanation of where the idea of residue comes from and how it is used in complex analysis.

# What is Residue?

Residue is a mathematical term in complex analysis that analyzes the behavior of closed contour integrals about sigular points. In order to understand the concept of residue, it is important that we have a basic understanding of complex integration. Specifically, the integral of the complex logarithm is really useful. Additionally, it is crucial to have a basic understanding of the Larentz series, which can be thought of as a generalization of the Taylor series to complex functions. For a more in depth explanation, I recommend Steve Brunton's videos on these topic. But for the sake of completeness, I will motivate a few of the core concepts here.

## The Complex Logarithm

let $ z $ be a variable in the complex plane.  
$ z = x + iy $  
let $ \omega = u + iv $, where $ u $ and $ v $ are real numbers 

Let the Complex Logarithm $ Log(z) $ be defined as the inverse of the complex expontial $ e^z $   
$ \omega = Log(z) \iff z = e^{\omega} $

Additionally, let z be expressed in polar form:
$ z = Re^{i\theta} $.   
$ e^{\omega} = e^{u}e{iv} = Re^{i\theta}$ where $ R = e^{u} $ and $ \theta = v $ 

By Euler's formula it becomes clear:  
$ \omega = Log(z) \iff z = e^{u}[cos(v) + isin(v)]$

additionally,  
$ u = log(R) = log(|z|) $    
$ v = \theta $

***Note, there is a syntactical distinction between the complex logarithm that we just defined $Log(z)$ and the real-valued logarithm $log(x)$.

So far, nothing that we have done is particularly interesting. However, we have only defined the logarithm for a particular case. $ \omega $ has infinitely many solutions because $ e^{i\theta} = e^{i(\theta + 2\pi n)} $ for any integer $ n $. Let's modify our definition of the complex logarithm to account for this.

$ z = x + iy =  Re^{i(\theta + 2\pi n)} $  
$ u = log(|z|) $   
$ v = \theta_0 + 2\pi n $  

Therefore Log(z) is a multi-valued function of z. It is defined as follows,
$$ Log(z) = log(|z|) + i(\theta_0 + 2\pi n)$$

For a pictorial representation, check out this video:
https://youtu.be/CpKD9aYEdms?si=YrW91PFdj8Ka0LdF&t=719

Consider the imaginary part of the $Log(z)$, as we move in a circle about the origin in the complex plane, $ im(Log(z)) $ will increase by $ 2\pi $ for each revolution. Mathematically speaking, Log(z) accumulates a phase of $ 2\pi $ for each revolution about the origin. This phase has the interesting property that the closed contour integral of $ Log(z) $ about the origin is not zero, but rather some multiple of $ 2\pi i $. This phase accumulation does not happen for single valued functions, because according to the fundamental theorem of calculus, the closed contour integral of any single-valued function must be zero.

## Complex Integration
Without going into too much detail about complex integration, the following example an complex antiderivative should be familiar to you from calculus.

$$\oint\limits_{C}(z-a)^n\,\mathrm{dz} = \frac{(z - a)^{n+1}}{n+1}\Big|_{z_0}^{z_1}$$  
For single valued functions, the closed contour integral above is always zero. since $z_0 = z_1$.
$$\oint\limits_{C}(z-a)^n\,\mathrm{dz} = \frac{(z - a)^{n+1}}{n+1}\Big|_{z_0}^{z_0} = 0$$  

While this is true for all cases $ n \neq - 1 $, at $n=-1$ things blow up!

But as you may recall, this is not unique to complex integration. In calculus, when we had to integrate an function of the form, $ \frac{1}{x} $, we used the natural logarithm. Let's use this intuition to motivate the concept of residue.

$$\oint\limits_{C}(z-a)^n\,\mathrm{dz} = Log(z-a)\Big|_{z_0}^{z_1} = 0$$ 
where $ z_0 $ and $ z_1 $ are the start and end points of the contour $ C $ for our multi-valued function $ Log(z) $.

We have already shown that one winding around the origin accumulates a phase of $ 2\pi i $. By a similar argument, a single winding around any single pole of a function will accumulate a phase of $ 2\pi i $. Therefore, $z_1$ and $z_0$ differe by $ 2\pi i $, and the closed contour integral of $ Log(z-a) $ about a pole is $ 2\pi i $.

Now we can define the residue of a function about a single pole $a_k$
$$Res(f, a_k) = \frac{1}{2\pi i}\oint\limits_{C}f(z)\,\mathrm{dz}$$

Loosely speaking, the residue of a complex function of the form $ \frac{r}{z-a_k} $ is the value $r$, where $r$ is constant.

Therefore, each of the poles $z_i$ in the expression 
$$ R(z) = \sum_{i=1}^{n} \frac{r_i}{z-z_i} $$
has an associated residue $ r_i $.